ArithmeticError

In [1]:
import numpy as np
import numpy.matlib as matlib
import math
import scipy as sp
from Point import Point
import TargetFunctions as tf
from ObjectveFunction import ObjectiveFunction


In [2]:
objective_fun = ObjectiveFunction(fun=TargetFunctions.sum_of_squares, bounds=None)

In [3]:
population = np.genfromtxt('temp.csv', delimiter=',')

In [4]:
population

array([[19.39373056, 14.96149086],
       [10.29780034, 11.96076647],
       [18.99306488, 10.36477986],
       [11.6056448 , 13.17459942],
       [11.8601143 , 11.88986946],
       [16.68553166, 18.7478698 ],
       [10.05651735, 16.99723376],
       [17.53650804, 19.05555081],
       [18.77951017, 18.71337614],
       [14.27300631, 16.55033747],
       [19.37662417, 17.2752235 ],
       [15.81615297, 17.93510205],
       [17.50372101, 12.68348095],
       [14.40245244, 10.98281379],
       [11.07065429, 18.23345358],
       [14.78677426, 17.33757039],
       [14.37804023, 17.32342269],
       [17.53936576, 13.38721638],
       [14.03572126, 13.54812382],
       [12.70830623, 19.58573533],
       [18.56115717, 13.77646943],
       [13.57265328, 16.10858023],
       [15.25682678, 10.81641685],
       [16.70470905, 14.422743  ],
       [17.64233524, 17.76641895],
       [15.08652902, 16.47169621],
       [18.14659095, 19.11161413],
       [13.86375196, 18.89441965],
       [17.41705432,

In [5]:
point_dim = 2
counteval = 200
sigma = 0.3
m = np.random.rand(point_dim)
lbd = 4 + math.floor(3*math.log(point_dim))
mu = lbd/2
mu = math.floor(mu)
weights = math.log(mu + 0.5) - np.array([math.log(i) for i in range(1, mu)])
weights = weights/np.sum(weights)
mueff = np.sum(weights)**2/np.sum(weights**2)
cc = (4 + mueff/point_dim) / (point_dim + 4 + 2*mueff/point_dim)
cs = (mueff + 2) / (point_dim + mueff+5)
c1 = 2 / ((point_dim + 1.3)**2 + mueff)
cmu = min(1-c1, 2 * (mueff-2 + 1/mueff) / ((point_dim + 2)**2 + mueff))
damps = 1 + 2 * max(0, math.sqrt((mueff - 1)/(point_dim + 1)) - 1) + cs
chiN = point_dim**0.5*(1-1/(4*point_dim)+1/(21*point_dim**2))

In [6]:
C = np.eye(point_dim)
invsqrtC = np.eye(point_dim)
pc = np.zeros(point_dim)
ps = np.zeros(point_dim)

In [7]:
generated_pop = m + sigma * np.random.multivariate_normal(np.zeros(point_dim), C, lbd)
generated_pop

array([[0.99229951, 0.39568687],
       [0.81144227, 0.85673862],
       [0.60280706, 0.67227931],
       [0.70103438, 0.00485372],
       [0.22673072, 0.81293349],
       [0.33877905, 0.35741382]])

In [8]:
points = np.array([Point(coordinates=i, objective_fun=objective_fun) for i in generated_pop])

In [9]:
fitnesses = np.array([p.value for p in points])
fitnesses

array([1.14122642, 1.39243962, 0.81533583, 0.49147276, 0.71226768,
       0.24251588])

In [26]:
index_order = fitnesses.argsort()
index_order

array([5, 3, 4, 2, 0, 1], dtype=int32)

In [28]:
fitnesses[index_order[::1]]

array([0.24251588, 0.49147276, 0.71226768, 0.81533583, 1.14122642,
       1.39243962])

In [33]:
np.array([p.coordinates for p in points])

array([[0.99229951, 0.39568687],
       [0.81144227, 0.85673862],
       [0.60280706, 0.67227931],
       [0.70103438, 0.00485372],
       [0.22673072, 0.81293349],
       [0.33877905, 0.35741382]])

In [30]:

generated_pop_sorted = generated_pop[index_order[::1]]
generated_pop_sorted

array([[0.33877905, 0.35741382],
       [0.70103438, 0.00485372],
       [0.22673072, 0.81293349],
       [0.60280706, 0.67227931],
       [0.99229951, 0.39568687],
       [0.81144227, 0.85673862]])

In [29]:
new_m = generated_pop_sorted.T[:,0:mu-1] @ weights
new_m

array([0.45063415, 0.24855236])

In [12]:
ps = (1-cs)*ps + math.sqrt(cs*(2-cs)*mueff) * invsqrtC @(new_m-m) / sigma
ps

array([ 0.39893428, -1.29164417])

In [13]:
hsig = np.linalg.norm(ps)/math.sqrt(1-(1-cs)**(2*counteval/lbd))/chiN < 1.4 + 2/(point_dim+1)

In [14]:
pc = (1-cc)*pc + hsig * math.sqrt(cc*(2-cc)*mueff) * (new_m-m) / sigma
pc

array([ 0.45157951, -1.46209555])

In [15]:
artmp = (1/sigma) * (generated_pop_sorted.T[:,0:mu-1] - matlib.repmat(m,mu-1,1).T)
artmp

array([[-0.00472452,  1.20279325],
       [-0.82902288, -2.00422323]])

In [16]:
C = (1-c1-cmu) * C + c1*(pc @ pc.T + (1-hsig)*cc*(2-cc)*C) + cmu * artmp @ np.diag(weights) @ artmp.T
C

array([[1.19254787, 0.34409119],
       [0.34409119, 1.23800791]])

In [17]:
sigma = sigma * math.exp((cs/damps)*(np.linalg.norm(ps)/chiN - 1))
sigma

0.30707984717060627

In [18]:
C = np.triu(C) + np.triu(C,1).T
C

array([[1.19254787, 0.34409119],
       [0.34409119, 1.23800791]])

In [19]:
D,B = np.linalg.eig(C)
D = np.array([math.sqrt(d)**(-1) for d in D])
D

array([1.07184352, 0.80061023])

In [20]:
invsqrtC = B @ np.diag(D) @ B.T

In [21]:
m = new_m

In [ ]:
0**(-1)

In [24]:
ZeroDivisionError

ZeroDivisionError